In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv('../input/amazoncom-employee-access-challenge/train.csv')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data["ACTION"].value_counts()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dc=DecisionTreeClassifier()

In [ ]:
from sklearn.model_selection import train_test_split

# Probability Distribution plot 

In [ ]:
import seaborn as sns
#Zoom in
for feature in data.columns:
    if feature !='ACTION':
        plt.figure(figsize=(9,6));
        sns.kdeplot(data[data['ACTION']==1][feature],label='Accepted',shade=True);
        sns.kdeplot(data[data['ACTION']==0][feature],label='Rejected',shade=True);
        plt.title('Distribution of '+feature+' variable');
        plt.xlabel(feature);
        plt.ylabel('Prob. Density');
        
        
    


# Correlation Heatmap

In [ ]:
sns.heatmap(data.corr())
plt.figure(figsize=(9,6));


features of dataset are not very correlated

# Target Class representation

In [ ]:
sns.countplot(data['ACTION'])

In [ ]:
print("The total % of ACTION taken are " + str(round(len(data[data['ACTION']==0])/len(data["ACTION"])*100,2))+"%")
print("The total % of not ACTION taken are " + str(round(len(data[data['ACTION']==1])/len(data["ACTION"])*100,2))+"%")

Data is hugely unbalanced as one class consists of 94% and the other has only 5% of total outcome.

# Target guided encoding

In [ ]:
dt=list()
data_preprocessed=data.copy()
for feature in data_preprocessed.columns:
    if feature!='ACTION':
        uniques=data_preprocessed[feature].unique()
        d=data.groupby(feature)
        mean_dict=dict()
        print(feature)
        for grps in d.groups:
            d1=d.get_group(grps).groupby('ACTION')
            try:
                zero=(d1.count().loc[0][1])
            except:
                zero=0
            try:
                ones=d1.count().loc[1][1]
            except:
                ones=0
            mean=round(zero/(zero+ones),2)
            #print("Mean for " + str(grps)+ " is "+str(mean))
            mean_dict[grps]=mean
        dt.append(mean_dict)
            
            
        
    

In [ ]:
data_preprocessed['RESOURCE']=data_preprocessed["RESOURCE"].replace(dt[0])


In [ ]:
data_preprocessed['MGR_ID']=data_preprocessed["MGR_ID"].replace(dt[1])
data_preprocessed['ROLE_ROLLUP_1']=data_preprocessed["ROLE_ROLLUP_1"].replace(dt[2])
data_preprocessed['ROLE_ROLLUP_2']=data_preprocessed["ROLE_ROLLUP_2"].replace(dt[3])
data_preprocessed['ROLE_DEPTNAME']=data_preprocessed["ROLE_DEPTNAME"].replace(dt[4])
data_preprocessed['ROLE_TITLE']=data_preprocessed["ROLE_TITLE"].replace(dt[5])
data_preprocessed['ROLE_FAMILY_DESC']=data_preprocessed["ROLE_FAMILY_DESC"].replace(dt[6])
data_preprocessed['ROLE_FAMILY']=data_preprocessed["ROLE_FAMILY"].replace(dt[7])
data_preprocessed['ROLE_CODE']=data_preprocessed["ROLE_CODE"].replace(dt[8])

In [ ]:
data_preprocessed.head()

# Model Building

In [ ]:
Y=data_preprocessed["ACTION"]
X=data_preprocessed.iloc[:,1:]
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=1,test_size=0.3)

In [ ]:
dc.fit(X_train,Y_train)

In [ ]:
Y_pred=dc.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
print(confusion_matrix(Y_test,Y_pred))

In [ ]:
Y_test.value_counts()

In [ ]:
print("The accuracy score for the model is " + str(accuracy_score(Y_test,Y_pred)))

In [ ]:
from sklearn.metrics import recall_score,precision_score

In [ ]:
print("The recall score for the model is " +str(recall_score(Y_test,Y_pred)))
print("The precision score for the model is " +str(precision_score(Y_test,Y_pred)))

In [ ]:
print("the F1 score for the model is "+str(f1_score(Y_test,Y_pred)))

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
print("The ROC_AUC score value of model is "+str(roc_auc_score(Y_test,Y_pred)))

In [ ]:
dt_prob=dc.predict_proba(X_test)[:,1]
dt_auc = roc_auc_score(Y_test, dt_prob)
# summarize scores
print('Decision tree: ROC AUC=%.3f' % (dt_auc))
# calculate roc curves
from sklearn.metrics import roc_curve
dt_fpr, dt_tpr, _ = roc_curve(Y_test, Y_pred)
# plot the roc curve for the model
plt.plot(dt_fpr, dt_tpr, marker='.', label='Decision Tree')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()